# Selección de Modelo + Tuning de Hiperparámetros  + Evaluación final del mejor modelo: k Fold Cross Validation Anidado

Nested CV - Raschka para sklearn 0.16 (modificado para que funcione en 0.18 y posterior)

Si bien está la versión 0.18 en otro archivo de Jupyter, ésta me parece más fácil de explicar a los chicos. 

In [1]:
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC



from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
np.random.seed(123)
random.seed(123)

## INTRODUCCIÓN 
Con el método " Separando el Test Set" al efectuar la comparación entre diferentes algoritmos o modelos, utilizamos el mismo Test Set para evaluar cada uno de los modelos. Ésto provocará que aprendamos "algo" del Test Set (ya que en base a él elegiríamos el mmodelo que mejor resultado dio), y la evaluación del mismo sería algo **optimista**.

Una forma de intentar resolver este problema  es ... aplicando Cross Validation sobre todo el DataSet Original, de tal forma de ir "rotando" el Test Set, para lo cual deberíamos repetir el proceso de la imagen anterior para diversos folds en los que habríamos dividido al DataSet original completo ( la imagen anterior sería sólo el primer fold de ese proceso).

Al loop más externo, el que divide al DataSet en diversos folds se lo suele denominar "loop externo" o "outer loop", al que se produce al dividir el Train también el folds, se lo suele denominar "loop interno"o "internal loop".

**El loop interno se utiliza sólo para determinar el algoritmo**. Un vez determinado el algoritmo que mejor funciona, entonces utilizaremos el loop externo para determinar el mejor valor de sus hiperparámetros y estimar una evaluación del modelo así elegido. 

## Los Datos

Esta vez voy a elegir un DataSet de verdad para que veamos un poco los tiempos que tarda.

In [2]:
# DataSet MNIST 5000 samples del total, 10% de todo el dataset real
# http://yann.lecun.com/exdb/mnist/

from mlxtend.data import mnist_data  # Puede ser que haya que instalar mlextend

X, y = mnist_data()
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=123,
                                                    stratify=y)

In [3]:
# Seleccionar:
cantidad_folds_outer= 5 
cantidad_folds_inner= 3

# Indicamos los modelos que usaremos
# creamos el diccionario cv_scores para guardar el valor de Accuracy en el outercv
cv_scores={'KNN':[],'DTREE':[],'RIDGE':[]}  # Diccionario para guardar resultados

In [4]:
gridcvs = {}

# Creamos los folds externos, outer
skfold = StratifiedKFold(n_splits=cantidad_folds_outer, shuffle=True, random_state=123)
skfold.get_n_splits(X_train, y_train)

# Loop externo, outer loop
fold_num = 1  # simplemente en número de fold en el que estamos, es para mostrar al hacer un print

for outer_train_idx, outer_valid_idx in skfold.split(X_train, y_train):
    
    # Desarrollamos cada uno de los modelos
    
    # Vamos con KNN **********************************************************************************
    # Instanciamos el estimador o 
    KNN=KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)
    param_grid_knn = [{'n_neighbors': list(range(1, 10)),'p': [1, 2]}]
    gcv_knn = GridSearchCV(estimator=KNN,
                        param_grid=param_grid_knn, 
                        scoring='accuracy',
                        n_jobs=-1,
                        cv=cantidad_folds_inner,
                        verbose=0,
                        refit=True)
    
    # Le hacemos Cross Validation en el inner loop, es decir hacemos GridSearchCV sobre el Train
    gcv_knn.fit(X_train[outer_train_idx], y_train[outer_train_idx]) # hacemos el fit para el GRidSearchCV que le corresponde
    y_pred = gcv_knn.predict(X_train[outer_valid_idx])
    acc = accuracy_score(y_true=y_train[outer_valid_idx], y_pred=y_pred)
    
    print('kNN | fold: ',fold_num,' | inner ACC %.2f%% | outer ACC %.2f%%' %
              (gcv_knn.best_score_ * 100, acc * 100))
    
    cv_scores['KNN'].append(acc)    # Guardo los valores del KNN en el outer
      
    # Vamos con DTREE **********************************************************************************
    DTREE=DecisionTreeClassifier(random_state=1)
    param_grid_DTREE =[{'max_depth': list(range(1, 10)) + [None],
                        'criterion': ['gini', 'entropy']}]
    
    gcv_dtree = GridSearchCV(estimator=DTREE,
                        param_grid=param_grid_DTREE, 
                        scoring='accuracy',
                        n_jobs=-1,
                        cv=cantidad_folds_inner,
                        verbose=0,
                        refit=True)
    
    gcv_dtree.fit(X_train[outer_train_idx], y_train[outer_train_idx]) # hacemos el fit para el GRidSearchCV que le corresponde
    y_pred = gcv_dtree.predict(X_train[outer_valid_idx])
    acc = accuracy_score(y_true=y_train[outer_valid_idx], y_pred=y_pred)
    
    print('DTREE | fold: ',fold_num,' | inner ACC %.2f%% | outer ACC %.2f%%' %
              (gcv_dtree.best_score_ * 100, acc * 100))
    
    cv_scores['DTREE'].append(acc)  # guardo los valores del DTREE
    
    # Vamos con RIDGE **********************************************************************************
    
    RIDGE=LogisticRegression(penalty='l2', max_iter=10000, tol=0.0001)
    param_grid_RIDGE =[{'C':[1e-6,1e-5,1e-4,0.001,0.01,0.1,1,10]}]
    
    
    gcv_ridge = GridSearchCV(estimator=RIDGE,
                        param_grid=param_grid_RIDGE, 
                        scoring='accuracy',
                        n_jobs=-1,
                        cv=cantidad_folds_inner,
                        verbose=0,
                        refit=True)
    
    gcv_ridge.fit(X_train[outer_train_idx], y_train[outer_train_idx]) # hacemos el fit para el GRidSearchCV que le corresponde
    y_pred = gcv_ridge.predict(X_train[outer_valid_idx])
    acc = accuracy_score(y_true=y_train[outer_valid_idx], y_pred=y_pred)
    
    print('RIDGE | fold: ',fold_num,' | inner ACC %.2f%% | outer ACC %.2f%%' %
              (gcv_ridge.best_score_ * 100, acc * 100))
    
    cv_scores['RIDGE'].append(acc)  # guardo los valores del DTREE
    
    fold_num=fold_num+1

kNN | fold:  1  | inner ACC 92.69% | outer ACC 91.38%
DTREE | fold:  1  | inner ACC 74.81% | outer ACC 75.75%
RIDGE | fold:  1  | inner ACC 89.31% | outer ACC 89.12%
kNN | fold:  2  | inner ACC 91.78% | outer ACC 94.62%
DTREE | fold:  2  | inner ACC 75.16% | outer ACC 74.38%
RIDGE | fold:  2  | inner ACC 89.44% | outer ACC 88.62%
kNN | fold:  3  | inner ACC 92.31% | outer ACC 94.50%
DTREE | fold:  3  | inner ACC 73.44% | outer ACC 80.00%
RIDGE | fold:  3  | inner ACC 88.78% | outer ACC 90.62%
kNN | fold:  4  | inner ACC 92.00% | outer ACC 93.88%
DTREE | fold:  4  | inner ACC 74.12% | outer ACC 77.00%
RIDGE | fold:  4  | inner ACC 89.97% | outer ACC 89.12%
kNN | fold:  5  | inner ACC 91.56% | outer ACC 93.38%
DTREE | fold:  5  | inner ACC 74.72% | outer ACC 78.38%
RIDGE | fold:  5  | inner ACC 88.88% | outer ACC 90.38%


### SELECCIÓN DE ALGORITMO

Buscamos el "mejor" modelo (algoritmo) **promediando** los valores obtenidos por cada modelo en  los folds del **outer**:

In [5]:
KNN_promedio=np.mean(cv_scores['KNN'])
KNN_desvio=np.std(cv_scores['KNN'])
DTREE_promedio= np.mean(cv_scores['DTREE'])
DTREE_desvio=np.std(cv_scores['DTREE'])
RIDGE_promedio=np.mean(cv_scores['RIDGE'])
RIDGE_desvio=np.std(cv_scores['RIDGE'])
print('KNN promedio en el outer: ',KNN_promedio*100, " +/- ", 2*KNN_desvio*100, " (95%)")
print('DTREE promedio en el outer: ', DTREE_promedio*100, " +/- ", 2*DTREE_desvio*100, " (95%)")
print('RIDGE promedio en el outer: ', RIDGE_promedio*100, " +/- ", 2*RIDGE_desvio*100, " (95%)")

KNN promedio en el outer:  93.54999999999998  +/-  2.3537204591879672  (95%)
DTREE promedio en el outer:  77.10000000000001  +/-  3.9287402561126394  (95%)
RIDGE promedio en el outer:  89.57499999999999  +/-  1.5620499351813346  (95%)


> Elegimos el **algoritmo** que mejor anduvo **pero no nos interesan los hiperparámetros utilizados en el inner**, éstos sólo sirvieron para comparar los algoritmos y elegir el que mejor anduvo.  El **inner** sólo nos sirvió para saber qué algoritmo vamos a usar.

> Ahora vamos a repetir el proceso de Cross Validation con todo el Train, **para el algoritmo elegido**, y **descubriremos sus mejores hiperparámetros**.

> Pero sí debemos utlizar **el mismo grid de hiperparámetros que utilizamos antes en el inner**. No podemos usar uno con más valores por ejemplo.

El algoritmo elegido es knn, fíjese que en el GridSearchCV gcv_knn tenemos cargada la grilla de parámetros que usamos anteriormente, así que  no necesitamos escribirla de nuevo!

In [6]:
gcv_knn

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='ball_tree', leaf_size=50,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                          'p': [1, 2]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

## Evaluación del Modelo / Hiperparámetros elegido

Entrenamos con fit al algoritmo elegido usando Cross Validation por eso usamos el GridSearchCV gcv_knn   

In [19]:
algoritmo_elegido=gcv_knn
algoritmo_elegido.fit(X_train, y_train)

AC_test=algoritmo_elegido.score(X_test, y_test)
AC_train=algoritmo_elegido.score(X_train, y_train)

print("AC_test: ", AC_test, "AC_train: ", AC_train)

AC_test:  0.944 AC_train:  1.0


In [15]:
algoritmo_elegido.best_params_

{'n_neighbors': 1, 'p': 2}